In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib.image as mpimg
import glob
import json
import os
import torch
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, sampler
import pandas as pd
import copy

In [2]:
dataset_path = "/Users/nduginets/PycharmProjects/master-diploma/GAN_to_box/test_data/isic_2018_boxes.csv"
frame = pd.read_csv(dataset_path)

,name,segm_x_0,segm_y_0,segm_x_size_0,segm_y_size_0,segm_x_1,segm_y_1,segm_x_size_1,segm_y_size_1,segm_x_2,...,streaks_x_size_12,streaks_y_size_12,streaks_x_13,streaks_y_13,streaks_x_size_13,streaks_y_size_13,streaks_x_14,streaks_y_14,streaks_x_size_14,streaks_y_size_14
0,0,0.060547,0.048387,0.763672,0.828446,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0,0.300781,0.351906,0.324219,0.280059,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0,0.066406,0.255132,0.841797,0.570381,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0,0.076172,0.177419,0.855469,0.523460,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0,0.250000,0.328446,0.447266,0.280059,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2589,1606,0.275391,0.342412,0.556641,0.341115,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2590,1606,0.525391,0.440986,0.207031,0.137484,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2591,1607,0.468750,0.443580,0.193359,0.159533,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2592,1607,0.435547,0.374838,0.189453,0.219196,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#  4 * 15
sz = 4 * 15
columns = frame.columns[1:]

attributes = [
    columns[i * sz: (i + 1) * sz]
    for i in range(6)
]
attributes

[Index(['segm_x_0', 'segm_y_0', 'segm_x_size_0', 'segm_y_size_0', 'segm_x_1',
        'segm_y_1', 'segm_x_size_1', 'segm_y_size_1', 'segm_x_2', 'segm_y_2',
        'segm_x_size_2', 'segm_y_size_2', 'segm_x_3', 'segm_y_3',
        'segm_x_size_3', 'segm_y_size_3', 'segm_x_4', 'segm_y_4',
        'segm_x_size_4', 'segm_y_size_4', 'segm_x_5', 'segm_y_5',
        'segm_x_size_5', 'segm_y_size_5', 'segm_x_6', 'segm_y_6',
        'segm_x_size_6', 'segm_y_size_6', 'segm_x_7', 'segm_y_7',
        'segm_x_size_7', 'segm_y_size_7', 'segm_x_8', 'segm_y_8',
        'segm_x_size_8', 'segm_y_size_8', 'segm_x_9', 'segm_y_9',
        'segm_x_size_9', 'segm_y_size_9', 'segm_x_10', 'segm_y_10',
        'segm_x_size_10', 'segm_y_size_10', 'segm_x_11', 'segm_y_11',
        'segm_x_size_11', 'segm_y_size_11', 'segm_x_12', 'segm_y_12',
        'segm_x_size_12', 'segm_y_size_12', 'segm_x_13', 'segm_y_13',
        'segm_x_size_13', 'segm_y_size_13', 'segm_x_14', 'segm_y_14',
        'segm_x_size_14', 'segm_y_

In [25]:

def noise_row(row, x_offset, y_offset, x_wide, y_wide):
    for j, attribute in enumerate(attributes):
        for i, a in enumerate(attribute):
            if "x_size" in a:
                row[j * sz + i] += x_wide
            elif "y_size" in a:
                row[j * sz + i] += y_wide
            elif "x" in a:
                row[j * sz + i] += x_offset
            elif "y" in a:
                row[j * sz + i] += y_offset
    return row

a = frame.iloc[2][1:]
print(a)
print(noise_row(a, 2, 2, 2, 2))

segm_x_0             0.066406
segm_y_0             0.255132
segm_x_size_0        0.841797
segm_y_size_0        0.570381
segm_x_1             0.000000
                       ...   
streaks_y_size_13    0.000000
streaks_x_14         0.000000
streaks_y_14         0.000000
streaks_x_size_14    0.000000
streaks_y_size_14    0.000000
Name: 2, Length: 360, dtype: float64
segm_x_0             2.066406
segm_y_0             2.255132
segm_x_size_0        2.841797
segm_y_size_0        2.570381
segm_x_1             2.000000
                       ...   
streaks_y_size_13    2.000000
streaks_x_14         2.000000
streaks_y_14         2.000000
streaks_x_size_14    2.000000
streaks_y_size_14    2.000000
Name: 2, Length: 360, dtype: float64
